<a href="https://colab.research.google.com/github/HardToGiveaName/ML-000/blob/main/Week02/week2_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week2 Homework

将 `target_encoding` 进行加速

## 原始代码

In [117]:
import numpy as np
import pandas as pd

def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [135]:
y = np.random.randint(2, size=(500, 1))
x = np.random.randint(10, size=(500, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
%timeit -n 1 target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 2.33 s per loop


## 代码含义
获取排除本行后，与本行关键字(x_name)相同的y_name样本的均值
`target_mean_v1` 函数接受3个参数  
- param1: shape为(N, 2)的DataFrame  
- param2: 非groupby使用的key_col_name  
- param3: groupby使用的key_col_name  

1. 逐行遍历, 将非本行的dataframe根据key_col_name进行groupby操作，并将 `mean` 及 `count` 聚合(agg)  
2. 将聚合后的mean数据，回写  

很明显，这段代码在遍历的过程中，执行了N次的groupby操作，时间复杂度较高

## 优化版本1
python代码级别的优化，遍历一次，将数据预取，最终在计算时进行排除

In [140]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] in value_dict.keys():
            count_dict[data.loc[i, x_name]] += 1
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
        else:
            count_dict[data.loc[i, x_name]] = 1
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [142]:
# Test correct
#ret1 = target_mean_v1(data, 'y', 'x')
#ret2 = target_mean_v2(data, 'y', 'x')
#diff = np.linalg.norm(ret1 - ret2)
#print(diff)
%timeit -n 1 target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 28 ms per loop


## 优化版本2

通过v2代码，可以看到在循环中大量引用 `data.loc[i, x_name]` 及 `data.loc[i, y_name]` 之处, 如果能够在使用前替换成栈变量，将会大大提速

In [270]:
def target_mean_v3(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    x_val_series = data.loc[:, x_name]
    y_val_series = data.loc[:, y_name]
    for i in range(data.shape[0]):
        x_tmp_index = x_val_series[i]
        y_tmp_index = y_val_series[i]
        if x_tmp_index in value_dict.keys():
            count_dict[x_tmp_index] += 1
            value_dict[x_tmp_index] += y_tmp_index
        else:
            count_dict[x_tmp_index] = 1
            value_dict[x_tmp_index] = y_tmp_index
    for i in range(data.shape[0]):
        result[i] = (value_dict[x_val_series[i]] - y_val_series[i]) / (count_dict[x_val_series[i]] - 1)
    return result

In [271]:
# Test correct
#ret1 = target_mean_v2(data, 'y', 'x')
#ret2 = target_mean_v3(data, 'y', 'x')
#diff = np.linalg.norm(ret1 - ret2)
#print(diff)
%timeit -n 1 target_mean_v3(data, 'y', 'x')

1 loop, best of 3: 10.7 ms per loop


## 优化版本3

偶然间发现，data.loc[:, x_name]的类型为 pd.Series  
通过 .values 方法将其转换为 np.ndarray, 将会大大提速

In [235]:
print(type(data.loc[:, 'x']))
print(type(data.loc[:, 'x'].values))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [272]:
def target_mean_v4(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    x_val_series = data.loc[:, x_name].values
    y_val_series = data.loc[:, y_name].values
    for i in range(data.shape[0]):
        x_tmp_index = x_val_series[i]
        y_tmp_index = y_val_series[i]
        if x_tmp_index in value_dict.keys():
            count_dict[x_tmp_index] += 1
            value_dict[x_tmp_index] += y_tmp_index
        else:
            count_dict[x_tmp_index] = 1
            value_dict[x_tmp_index] = y_tmp_index
    for i in range(data.shape[0]):
        result[i] = (value_dict[x_val_series[i]] - y_val_series[i]) / (count_dict[x_val_series[i]] - 1)
    return result

In [251]:
# Test correct
#ret1 = target_mean_v3(data, 'y', 'x')
#ret2 = target_mean_v4(data, 'y', 'x')
#diff = np.linalg.norm(ret1 - ret2)
#print(diff)
%timeit -n 1 target_mean_v4(data, 'y', 'x')

1 loop, best of 3: 993 µs per loop


## 优化版本4
将数据结构改造成cython版本
np.ndarray -> cnp.ndarray
python.dict -> cython.dict

In [171]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp
import pandas as pd

def target_mean_v5(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    cdef:
        #cdef cnp.ndarray[cnp.int_t, ndim=1] result = np.zeros(data.shape[0], dtype='int32')
        dict value_dict = {}
        dict count_dict = {}
        cnp.ndarray[cnp.int_t] x_val_series = data.loc[:, x_name].values
        cnp.ndarray[cnp.int_t] y_val_series = data.loc[:, y_name].values
    for i in range(data.shape[0]):
        x_tmp_index = x_val_series[i]
        y_tmp_index = y_val_series[i]
        if x_tmp_index in value_dict.keys():
            count_dict[x_tmp_index] += 1
            value_dict[x_tmp_index] += y_tmp_index
        else:
            count_dict[x_tmp_index] = 1
            value_dict[x_tmp_index] = y_tmp_index
    for i in range(data.shape[0]):
        result[i] = (value_dict[x_val_series[i]] - y_val_series[i]) / (count_dict[x_val_series[i]] - 1)
    return result

In [267]:
# Test correct
#ret1 = target_mean_v4(data, 'y', 'x')
#ret2 = target_mean_v5(data, 'y', 'x')
#diff = np.linalg.norm(ret1 - ret2)
#print(diff)
%timeit -n 100 target_mean_v5(data, 'y', 'x')

100 loops, best of 3: 864 µs per loop


In [276]:
%timeit -n 1 target_mean_v1(data, 'y', 'x')
%timeit -n 1 target_mean_v2(data, 'y', 'x')
%timeit -n 100 target_mean_v3(data, 'y', 'x')
%timeit -n 100 target_mean_v4(data, 'y', 'x')
%timeit -n 100 target_mean_v5(data, 'y', 'x')

1 loop, best of 3: 2.34 s per loop
1 loop, best of 3: 26.8 ms per loop
100 loops, best of 3: 9.28 ms per loop
100 loops, best of 3: 940 µs per loop
100 loops, best of 3: 789 µs per loop


## 优化效果
纯python代码，由2.34s优化到940us，速度提升 2489倍
cython代码，由940us优化到789us，速度提升1.19倍，cython版本仍有优化空间，待继续探索如何更好地使用cython提升效率